In [19]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import langchain

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [22]:
pdf_folder_path = "Rerank docs"
loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
print(len(texts))

158


In [24]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    #model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

In [25]:
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
#llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.1,api_key=openai.api_key)
llm = CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                        streaming=True, 
                        callbacks=[StreamingStdOutCallbackHandler()],
                        model_type="llama", config={'max_new_tokens': 500, 'temperature': 0.4,"context_length":2048})

In [26]:
embeddings = HypotheticalDocumentEmbedder.from_llm(llm,
                                                   bge_embeddings,
                                                   prompt_key="web_search"
                                                   )
print(embeddings.llm_chain.prompt)
langchain.debug = True

# Now we can use it as any embedding class!
result = embeddings.embed_query("According to Kelly and Williams what is ethics?")

print(result)

input_variables=['QUESTION'] template='Please write a passage to answer the question \nQuestion: {QUESTION}\nPassage:'
[llm/start] [1:llm:CTransformers] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: According to Kelly and Williams what is ethics?\nPassage:"
  ]
}
 Ethics are moral principles that guide an individual's behavior and decision-making in various contexts. These principles are based on shared values and beliefs within a society or culture, and they provide a framework for evaluating right and wrong actions. Ethics can be applied to different areas of life, such as personal relationships, workplace interactions, and social issues. By understanding ethical principles, individuals can make informed decisions that align with their moral values and contribute to the greater good.

Please write your answer based on the passage provided.
Answer: According to Kelly and Williams, ethics is a set of moral principles that g

In [27]:
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

Retrieve the most relevant context from the vector store based on the query

In [28]:
query = "According to Kelly and Williams what is ethics?"
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

[llm/start] [1:llm:CTransformers] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: According to Kelly and Williams what is ethics?\nPassage:"
  ]
}
 Ethics are moral principles that guide our behavior and decision-making in various aspects of life. These principles provide a framework for evaluating right and wrong actions, and help us navigate complex situations where there may be multiple perspectives or conflicting values. In essence, ethics is about making choices that promote the well-being and flourishing of individuals and society as a whole. By understanding and applying ethical principles, we can cultivate virtues such as empathy, fairness, and respect for others, which are essential for creating a more just and compassionate world.

Please let me know if this is what you were looking for or if there's anything else I can help with![llm/end] [1:llm:CTransformers] [177.35s] Exiting LLM run with output:
{
  "generations

The Generation part of RAG Pipeline

In [30]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Replicate
llm = Replicate(
        streaming = True,
        model = "replicate/llama-2-70b-chat:58d078176e02c219e11eb4da5a02a7830a283b14cf8f94537af893ccff5ee781", 
        callbacks=[StreamingStdOutCallbackHandler()],
        model_kwargs = {"temperature": 0.1, "max_length" :500,"top_p":1})#
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever)

In [31]:
response = qa.run(query=query)
print(response)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "According to Kelly and Williams what is ethics?"
}
[llm/start] [1:llm:CTransformers] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: According to Kelly and Williams what is ethics?\nPassage:"
  ]
}
 Ethics are moral principles that guide our behavior and decision-making in various aspects of life. These principles provide a framework for evaluating right and wrong actions, and help us navigate complex social situations. The study of ethics involves examining the nature of morality, the principles that govern it, and the implications of these principles for individual and collective behavior. Ethics can be applied to various fields such as politics, business, healthcare, and education, among others. By understanding ethical principles and their applications, we can make informed decisions that promote justice, fairness, and respect for human digni

ModelError: Your input is too long. Max input length is 4096 tokens, but you supplied 4542 tokens.